# ECE 111 - A Summary of the Bitcoin Network

## Motivation

Bitcoin has gained a lot of press coverage over the last few months. It is oftetimes touted as either a digital currency or store of value, but it highly debated whether it will fill that role anytime in the near future. Bitcoin can be described as a "decentralized" payment system that does not require communication with a central authority to verify transactions but rather relies on the "work" of an uncorrdinated set of Bitcoin miners to verify transactions. The technology behind the Bitcoin Network is applicable to other applications than just digital currency. 

As engineers, it is vital to understand emerging technologies. The IEEE Computer Society predicts that 2018 will be a big year for blockchain technologies. [1] Therefore it is useful to understand on a basic level how and why the Bitcoin blockchain works. We study Bitcoin as an example to uncover the backbone of a decentralized network, but many other "coins" either use the same technology or borrow ideas from Bitcoin, and there are sure to be further projects in the coming years. We will skip over minor details that are outlined in more detail in Satashi Nakamoto's whitepaper [2] and focus on how transactions are verified without a central entity such as government or corporations.

## Bitcoin Core

_At it's core, Bitcoin Core is a decentralized payment system. A pool of miners ensure the integrity of the blockchain and users compensate the miners through transaction fees._

First, here are some useful definitions:
- *Node* - a computer that connects to the Bitcoin network.
- *Proof of Work* - The algorithm that miners use to verify a transaction.
- *SHA256* - The hashing algorithm used in Proof of Work. It maps a message of size less than 2^64 bits and unique "block" (this is different that a bitcoin block described below) of size 256 bits. It is extremely difficult to compute an inverse-hash, but if one knows the message he will get the same output every time. Thus, this a good way to encode information.
- *Nonce* - The "golden ticket" for Bitcoin miners. It is a 32-bit value that is the output of the Proof of Work algorithm. This is described in more detail under Proof of Work.
- *Block* - A collection of transcations up to (at the time of writing) 1 MB in size. To verify a transaction on the Bitcoin network though Proof of Work, all the information needed is in the block-header. A block-header is 80bytes or 640bits.
- *Bitcoin Core* - a piece of software that all nodes need to download in order to connect to the bitcoin network which allows users to send/recieve Bitcoin.

A blockchain is a chain of blocks dating back to the very first Bitcoin block. In order to use the Bitcoin network, one must download bitcoin core which includes a copy of the entire blockchain (though in practice most users store the majority of the data remotely). The blockchain is also known as a public ledger since it provides proof of reciept and the timestamp for every transaction ever made in Bitcoin. 

Miners compete to process transactions. For our purposes, the first to find a nonce that satisfies the target appends that block to the blockchain. In other words, miners compete to establish consensus over which transaction is allowed to go through and which is left in the queue. On average 1 block is mined every 10 minutes. The bitcoin protocol adjusts the difficulty target to keep it that way. Miners compete to mine that block, and if they are successful they are rewarded. 

At the time of writing the processing power needed to mine a block in time averages `(1,590,896,927,258 2^32 Hashes to mine a block)/(8,481,426,187 Giga Hash/second) = 805seconds` [3]. In December, it took even longer since it took 11,388,083,790 Giga Hashes/second to mine 1 block [4]. Obviously, this is a huge amount of computing power and the chance of one computer successfully finding a nonce in a resonable amount of time even with the best hardware available is almost 0. Because of the difficulty of Proof of Work, nowadays computers usually work in "pools" where they share computing power and also share the rewards. 

Proof of Work expends resources (power), which is why miners need to be compensated for their work in order to sustain the network. It is unfeasable for bad actors to try to tamper with the blockchain by spamming the network because they expend resources and get no return, while miners are compensated for their work and presumably use some of that compensation build better machines. 

The incentive to mine is that miners are rewarded a certain number of newly minted coins for processing the first
transaction of a block. The bitcoin reward starts at 25 bitcoins. Every 200,000 blocks that number is halved. Miners are also compensated through transaction fees provided by nodes as they send their Bitcoin across the network. The "highest-bidders", or nodes with the highest transaction fees, are the first to be included in the next block. 

For example, by going to a blockchain explorer like https://blockchain.info/ we see that there are (at the time of writing) 504,717 blocks that have been mined. That means that the current reward is `25/Floor[504,717/200,000] = 12.5 BTC`. 

After completing the Proof of Work algorithm, signaled by finding the correct nonce, and in effect "mining" a Bitcoin, miners append the block they mine to the blockchain. 

## A closer look at Proof of Work

### First, a quick review of SHA-256

SHA-256 can be thought of as an algorithm with 4 steps. It takes as input a message. The message can be anything you like. But the message is represented in bits. For example, strings are sometimes used as a message but they are first decoded into bits. For future reference when we say SHA256(message) in psuedocode, message can be from 1 bit to 2^64 -1 bits. 

- Step 1: Pre-process the message by padding properly and appending length as a 64-bit unsigned. For example if the message is 900 bits, it should look like this:
<img src="sha256-7.png" width="60%" height="60%">
[6]
- Step 2: Process the message by dividing the message into 512-bit "blocks" then run through the 80-steps described in the lecture slides.
- Step 3: Let Output = {H0,H1,H2,H3,H4} which is our 256-bit hash value.

### In more detail, the steps are as follows:
<img src="sha256-1.png" width="60%" height="60%">
<img src="sha256-2.png" width="60%" height="60%">
<img src="sha256-3.png" width="60%" height="60%">
<img src="sha256-4.png" width="60%" height="60%">
<img src="sha256-5.png" width="60%" height="60%">
<img src="sha256-6.png" width="60%" height="60%">

[7]

For an alternate explanation look at the psuedocode on wikipedia [5].

### Proof of Work

The premise behind Proof of Work is that miners
“convert” electricity to bitcoin by performing the **Proof of Work algorithm**, simply refered to as Proof of Work, that awards
them with newly minted coins and transaction fees. Electricity costs money, so the net effect is
nodes are incentivized to act honestly (i.e. not congest the network, but actually verify
transactions). Each pool of miners verify a collection of transactions averaging every 10 minutes. A
collection of transactions is also called a block (currently set to 1MB worth of transactions, but this is
a soft cap i.e. it can be raised higher). 


Proof of Work acts on the block header. The block header looks like this:
![Image](hash_header.png) [8]

Notice that the block header is a total of 640-bits. All the fields are self-explanitory except for what is called a "nonce". A nonce is a number which, when included in the block header, causes the following expression to be true: 
```SHA256(SHA256(block_header)) < target```

If the above expression evaluates to true, then the block is published to the blockchain. If it evaluates to false, then we increment nonce and try again.

In [1]:
import hashlib, struct, codecs
import time

#calculate target beforehand. This is handled by bitcoin core and not miners.
bits = 0x19015f53

exp = bits >> 24

mant = bits & 0xffffff

target_hexstr = '%064x' % (mant * (1<<(8*(exp - 3))))

target = codecs.decode(target_hexstr, "hex")

# Define the block header except for nonce
ver = 2
prev_block = "000000000000000117c80378b8da0e33559b5997f2ad55e2f7d18ec1975b9717"
mrkl_root = "871714dcbae6c8193a2bb9b2a69fe1c0440399f38d94b3a0f1b447275a29978a"
time_ = 0x53058b35 # 2014-02-20 04:57:25
bits = 0x19015f53
header = ( struct.pack("<L", ver) + codecs.decode(prev_block, "hex")[::-1] +
              codecs.decode(mrkl_root, "hex")[::-1]) 

In [2]:
#Based off of code described here http://www.righto.com/2014/02/bitcoin-mining-hard-way-algorithms.html#ref3
def PoW(block_header,target):
    nonce = 850000000 # I cheated. Normally nonce starts at 0 but I set it close to the nonce.
    #Run Proof of Work
    while nonce < 0x100000000: #while nonce < 2^32
        block_header = (header + struct.pack("<LLL", time_, bits, nonce)) #append nonce.
        
        hash = hashlib.sha256(hashlib.sha256(block_header).digest()).digest() 
        if hash[::-1] < target:
            print('success')
            print ("Time elapsed:", time.time() - start_time, "seconds")
            print ("The golden nonce is: ", nonce)
            break
            
        nonce += 1

In [3]:
start_time = time.time()

PoW(header,target)


success
Time elapsed: 14.386127233505249 seconds
The golden nonce is:  856192328


As one can see, most of the complexity of the algorithm comes from the sha256 function. We can also see that a for loop is incredibly inefficient seeing that even when we started at nonce = 8 million, that it took a good amount of time to  long time. In the project, the difficulty will be adjusted to keep it so that even the less optimized designs are able to complete the mining process in a reasonable amount of time. Hopefully you guys will design a FPGA can do this much faster.

### A closer look at ```SHA256(SHA256(block_header)) < target```

Starting inward, we have ```SHA256(block_header)```.
Due to the nature of the sha256 algorithm, what is actually happening here is that we first break the block_header into two "blocks" of 64-bytes (=512 bits) and 16-bytes. 64-bytes = 512 bits, so the former remains constant and we have to pad the 16-byte=128bits and append the length of the block_header. Notice that there are some obvious optimizations to be made already. First off, the 64-bytes are constant, so we can run the sha256 algorithm on that portion alone and then store that value in a variable. Then reuse that variable over and over instead of running sha256 on it every time. Notice also that the length is constant, so one can define a parameter at the top of the file with the length, i.e. `parameter len_header = 64'd640`. 

Assuming that the rest of the inward SHA256 function is accounted for, then we now have an outer function `SHA256_2(SHA_256_output1)` to compute. Again since we know that the sha256_output1 will have length = 256 bits we can define a parameter at the top like so, `len_sha256_output1 = 64'256`. Since we do not know anything about the 
sha_256_output1 (remember that outputs to sha256(...) is essentially random), we will have to use a sha256 algorithm like one in the previous projects, i.e we know nothing about the message.

For best results, I suggest pipelining both the inner and outer sha256 algorithms. Good luck on the project.

[1] : Prediction by IEEE - https://www.computer.org/web/pressroom/top-technology-trends-2018

[2] : Nakamoto's whitepaper - https://bitcoin.org/bitcoin.pdf

[3] : https://bitcoin.stackexchange.com/questions/52032/how-much-time-in-average-does-it-take-for-an-simple-pc-to-create-block

[4] : https://bitcoinwisdom.com/bitcoin/difficulty

[5] : https://en.wikipedia.org/wiki/SHA-2

[6],[7] : Professor Lin's slides, to be uploaded. This will be updated when they are posted on the website.

[8] : http://digitalcommons.calpoly.edu/cgi/viewcontent.cgi?article=1285&context=eesp